# use_TOOL_CALL_handler

In [1]:
# Create a function that returns a dictionary of tools.
# Note that 

def create_tools_dict():
    doc_titles=["MICHELIN Guide Singapore 2016: Where to Find the Best Chicken Rice in Singapore"]
    doc_keywords=["Singapore","Chicken Rice","Food"]
    tools_dict={ 
        "google":{
                "type": "function",
                "function": {
                    "name": "google",
                    "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                    "arguments": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                            }
                        },
                        "required": ["search_query"]
                    }
                }
            },
        "retrieval":{
            "type": "function",
            "function": {
                "name": "retrieval",
                "description": f"""
                    The `retrieval` function is a powerful tool that allows the AI to access articles outside of its knowledge domain from external sources. 
                    The external articles are stored in an archive and organised by <titles>:\n{{ titles: [{doc_titles}] }}
                    and <keywords>:
                    {{ keywords: [{doc_keywords}] }}
                    **IMPORTANT**: Use this tool when any of the <titles> or <keywords> may be relevant to user's question.
                    The \'search_query\' parameter should be crafted in a way that it returns the most precise result based on the conversation context.
                """,
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": """The most effective search query for semantic search that will return the most precise result."""
                        }
                    },
                    "required": ["search_query"]
                }
            }
        },
        "text": {
            "type": "function",
            "function": {
                "name": "text",
                "description": "The 'text' function is the default catch-all function returned when none of the other tools are applicable.",
                "arguments": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "The user's message."
                        }
                    },
                    "required": ["message"]
                }
            }
        }
    }
    return tools_dict

In [2]:
from gai.persona.fsm.handlers.use_TOOL_CALL_handler import use_TOOL_CALL_handler
from gai.ttt.client.ttt_client import TTTClient
ttt=TTTClient({
    "type": "ttt",
    "url": "http://localhost:12031/gen/v1/chat/completions"
})
from gai.lib.dialogue.MonologueMessageBuilder import MonologueMessageBuilder
messages = MonologueMessageBuilder(
    ).AddUserMessage("What is the current time in Singapore?"
    ).AddAssistantMessage(
    ).BuildRoleMessages()

handler=use_TOOL_CALL_handler()
handler.ttt=ttt
handler.tools_dict=create_tools_dict()
handler.tool_name="google"
handler.monologue_messages=messages
handler.max_new_tokens=300
handler.max_tokens=300
handler.on_TOOL_CALL()

from gai.lib.common.notebook import highlight
highlight(handler.content)

In [3]:
from gai.persona.fsm.handlers.use_TOOL_CALL_handler import use_TOOL_CALL_handler
from gai.ttt.client.ttt_client import TTTClient
ttt=TTTClient({
    "type": "ttt",
    "url": "http://localhost:12031/gen/v1/chat/completions"
})
from gai.lib.dialogue.MonologueMessageBuilder import MonologueMessageBuilder
messages = MonologueMessageBuilder(
    ).AddUserMessage("Where can I find the best hainanese chicken rice in Singapore?"
    ).AddAssistantMessage(
    ).BuildRoleMessages()

handler=use_TOOL_CALL_handler()
handler.ttt=ttt
handler.tools_dict=create_tools_dict()
handler.tool_name="retrieval"
handler.monologue_messages=messages
handler.max_new_tokens=300
handler.max_tokens=300
handler.on_TOOL_CALL()

from gai.lib.common.notebook import highlight
highlight(handler.content)